**Задание**

Для каждой страны необходимо подсчитать количество дней, когда количество умерших было больше, чем в среднем по миру, за все время наблюдений. Ответом к задаче будет топ-10 стран с наибольшим показателем количества таких дней.

Важные моменты
* Считаются дни, когда было строго больше среднего
* Разные регионы страны вначале необходимо совместить, чтобы статистика считалась верно
* Доработка данных уже после запроса на Python вполне возможна, если это требуется
* Не обязательно делать ровно один агрегационный запрос — можно сделать серию запросов. Главное, чтобы по итогу был подсчитан запрашиваемый набор данных

Результатом должен быть файл `result.json` в следующем формате:
```
{
    "<country 1>": "<number of days>",
    "<country 2>": "<number of days>",
    ...
    "<country 10>": "<number of days>"
}
```

Более подробную информацию про составление агрегационных пайплайнов в MongoDB можно найти в [официальной документации](https://docs.mongodb.com/manual/core/aggregation-pipeline/).

In [1]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
world = client.covid.world

In [2]:
from pprint import pprint

def p(x):
    pprint(list(x))

In [18]:
# считаем среднее по миру
query = world.aggregate([{"$group": {"_id": {"country": "$country",
                                             "date": "$date"},
                                     "deaths": {"$sum": "$deaths"}}}])

In [19]:
res = [doc for doc in query]

In [21]:
res[0]

{'_id': {'country': 'Ireland', 'date': datetime.datetime(2020, 7, 5, 0, 0)},
 'deaths': 1741}

In [22]:
total_deaths = 0

for country in res:
    total_deaths += country['deaths']

In [24]:
avg_death = total_deaths / len(res)
avg_death

1027.9561092447432

In [26]:
query = world.aggregate([{'$group': {'_id': {'country': '$country',
                                             'date': '$date'},
                                     'deaths': {'$sum': '$deaths'}}},
                         {'$match': {'deaths': {'$gt': avg_death}}},
                         {'$group': {'_id': {'country': '$_id.country'},
                                     'count': {'$sum': 1}}},
                         {'$sort': {'count': -1}},
                         {'$limit': 10}])

In [28]:
res = [doc for doc in query]
res

[{'_id': {'country': 'China'}, 'count': 151},
 {'_id': {'country': 'Italy'}, 'count': 120},
 {'_id': {'country': 'Iran'}, 'count': 115},
 {'_id': {'country': 'Spain'}, 'count': 113},
 {'_id': {'country': 'France'}, 'count': 109},
 {'_id': {'country': 'US'}, 'count': 108},
 {'_id': {'country': 'United Kingdom'}, 'count': 106},
 {'_id': {'country': 'Netherlands'}, 'count': 102},
 {'_id': {'country': 'Germany'}, 'count': 100},
 {'_id': {'country': 'Belgium'}, 'count': 99}]

In [29]:
final_result = {}

for country in res:
    final_result[country['_id']['country']] = country['count']

In [30]:
final_result

{'China': 151,
 'Italy': 120,
 'Iran': 115,
 'Spain': 113,
 'France': 109,
 'US': 108,
 'United Kingdom': 106,
 'Netherlands': 102,
 'Germany': 100,
 'Belgium': 99}

In [32]:
import json

with open("result.json", 'w') as f:
    f.write(json.dumps(final_result))

In [33]:
! cat result.json

{"China": 151, "Italy": 120, "Iran": 115, "Spain": 113, "France": 109, "US": 108, "United Kingdom": 106, "Netherlands": 102, "Germany": 100, "Belgium": 99}